In [93]:
#!/usr/bin/python3

from login_details import engine1
from pandas import DataFrame
import pandas as pd

In [115]:
# get data from stoppage_id db
with engine1.connect() as con:
    stoppage_hist_query = con.execute("""
    SELECT id, stoppage_id, datetime
    FROM dg_ln_stoppages_raw
    ORDER BY datetime ASC
    LIMIT 100
    """)
    con.close()

In [116]:
# get data from stoppage_string db
with engine1.connect() as conn:
    stoppage_str_query = conn.execute("""
    SELECT fault_id, fault_str, line_stopped, primary_reason
    FROM dg_stoppage_strings
    ORDER BY fault_id ASC
    """)
    con.close()

In [117]:
# convert database instances to DataFrame

# converting stoppage string instance
stoppage_str_df = DataFrame(stoppage_str_query.fetchall())
stoppage_str_df.columns = stoppage_str_query.keys()
stoppage_str_df.set_index('fault_id')

# converting stoppage history instance
stoppage_history_df = DataFrame(stoppage_hist_query.fetchall())
stoppage_history_df.columns = stoppage_hist_query.keys()


In [119]:
# create the "occurred_at" column with the correct datetime format
stoppage_history_df['occurred_at'] = pd.to_datetime(stoppage_history_df['datetime'].dt.strftime('%d-%m-%Y %H:%M:%S'))

# remove original datetime
del stoppage_history_df['datetime']
del stoppage_history_df['id']

stoppage_history_df.sort_values(by='occurred_at')

,stoppage_id,occurred_at
0,0,2021-10-19 06:49:43
1,64,2021-10-19 06:55:17
2,49,2021-10-19 06:55:19
3,1,2021-10-19 06:55:51
4,59,2021-10-19 06:55:53
...,...,...
95,59,2021-10-19 10:31:58
96,0,2021-10-19 10:32:18
97,42,2021-10-19 10:42:08
98,0,2021-10-19 10:43:56


In [122]:
for id, stoppage_id in stoppage_history_df.iterrows():
    print(occured_at)

stoppage_id                      0
occurred_at    2021-10-19 06:49:43
Name: 0, dtype: object
stoppage_id                     64
occurred_at    2021-10-19 06:55:17
Name: 1, dtype: object
stoppage_id                     49
occurred_at    2021-10-19 06:55:19
Name: 2, dtype: object
stoppage_id                      1
occurred_at    2021-10-19 06:55:51
Name: 3, dtype: object
stoppage_id                     59
occurred_at    2021-10-19 06:55:53
Name: 4, dtype: object
stoppage_id                      0
occurred_at    2021-10-19 06:55:57
Name: 5, dtype: object
stoppage_id                     44
occurred_at    2021-10-19 06:57:30
Name: 6, dtype: object
stoppage_id                      0
occurred_at    2021-10-19 07:00:12
Name: 7, dtype: object
stoppage_id                     59
occurred_at    2021-10-19 07:13:48
Name: 8, dtype: object
stoppage_id                      1
occurred_at    2021-10-19 07:13:50
Name: 9, dtype: object
stoppage_id                      0
occurred_at    2021-10-19 07:14:04


In [120]:
# print rows based on stoppage_id value
# stoppage_history_df.loc[stoppage_history_df['stoppage_id']==0]

,stoppage_id,occurred_at
0,0,2021-10-19 06:49:43
5,0,2021-10-19 06:55:57
7,0,2021-10-19 07:00:12
10,0,2021-10-19 07:14:04
14,0,2021-10-19 07:15:52
16,0,2021-10-19 07:18:20
18,0,2021-10-19 07:23:56
20,0,2021-10-19 07:30:14
25,0,2021-10-19 07:34:14
31,0,2021-10-19 08:02:46
